In [ ]:
# file สำหรับตรวจคำตอบ
# ในกรณีที่มีการสร้าง function อื่น ๆ ให้ระบุว่า input-output คืออะไรด้วย
'''
ชื่อ_รหัส(ex: ธนวัฒน์_6461)
1. ณพวิทย์ 6519
2. อนัญญา 6557
3.
'''

### Import Library

In [2]:
import roboticstoolbox as rtb
import numpy as np

from spatialmath import SE3
from math import pi

from HW3_utils import *
from FRA333_HW3_6519_6557 import *

### Question 1

In [3]:
#define link length
d1,a2, = 0.0892,-0.425

T_3_e = SE3(-0.39243,-0.093,0.109)*SE3(- 0.082,0,0)*SE3.RPY(0,-pi/2,0)

In [4]:
# Find MDH Parameter using Robotics toolbox
robot = rtb.DHRobot(
    [
        rtb.RevoluteMDH(d = d1, offset=pi),
        rtb.RevoluteMDH(alpha=pi/2),
        rtb.RevoluteMDH(a= a2)
    ],
    tool = T_3_e,
    name = "RobotHW3"
    )
print(robot)

DHRobot: RobotHW3, 3 joints (RRR), dynamics, modified DH parameters
┌────────┬───────┬────────────┬────────┐
│  aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├────────┼───────┼────────────┼────────┤
│    0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│    0.0 │ 90.0° │         q2 │    0.0 │
│ -0.425 │  0.0° │         q3 │    0.0 │
└────────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘



### Question 1 Prove

In [11]:
# Random q
q = np.random.rand(3)*np.pi

# take jacobian fron the matrix above
J = robot.jacob0(q)

# Print it and compare with Jacobian matrix that you calculate 
J_e = endEffectorJacobianHW3(q)
print(f"Jacobian from calculate\n {J_e.round(6)}")
print("-----------------------------------")
print(f"Jacobian from prove\n {J.round(6)}")




# R,P,R_e,p_e = FKHW3(q)
# print(f"Position = {p_e}")
# print(f"Orientation =\n {R_e}")
# print(robot.fkine(q))

Jacobian from calculate
 [[-0.05594  -0.004941 -0.142792]
 [-0.137186  0.013252  0.382989]
 [ 0.       -0.100341 -0.258191]
 [ 0.       -0.936995 -0.936995]
 [ 0.       -0.349344 -0.349344]
 [ 1.        0.        0.      ]]
-----------------------------------
Jacobian from prove
 [[-0.05594  -0.004941 -0.142792]
 [-0.137186  0.013252  0.382989]
 [ 0.       -0.100341 -0.258191]
 [ 0.       -0.936995 -0.936995]
 [-0.       -0.349344 -0.349344]
 [ 1.        0.        0.      ]]


### Question 2 Prove

In [14]:
# Random Q
q = np.random.rand(3)*np.pi
print(f"Random q : {q}")
# Check singularity from robotics toolbox
J = robot.jacob0(q)
# Get only 3x3 matrix determinant used only squared matrix
submatrix_J = J[0:3,:]
# Find determinant
determinant = np.linalg.det(submatrix_J)
prove_singular = abs(determinant) < 0.001
print(f"Determinant for proving : {determinant}")
print(f"ผลลัพธ์ที่ได้จาก Robotics toolbox : {prove_singular}")


#Check from calculate
prove_calculate = checkSingularityHW3(q)
print(f"ผลลัพธ์ที่ได้จากที่คำนวณ : {bool(prove_calculate)}")

#Compare prove_singular and prove_original is it the same
if (prove_calculate == prove_singular):
    print(f"ผลลัพธ์ทั้งสองมีค่าตรงกัน : {True}")
else :
    print (False)



Random q : [1.82480747 1.72782201 0.53950574]
Determinant for proving : -0.06079564763143089
ผลลัพธ์ที่ได้จาก Robotics toolbox : False
ผลลัพธ์ที่ได้จากที่คำนวณ : False
ผลลัพธ์ทั้งสองมีค่าตรงกัน : True


### Question 3 Prove


In [21]:
def prooftau(q:list[float], w:list[float])->list[float] :
    
    # Get a tau from function computeEffortHW3(q,w)
    calculate_tau = computeEffortHW3(q,w)
    print(f"Calculate tau is {calculate_tau.round(6)}")

    # do a jacobian matrix using robotics toolbox
    J = robot.jacob0(q)
    subJ_T = np.transpose(J)
    tau_rtb = np.dot(subJ_T,w)
    print(f"RTB tau is {tau_rtb.flatten().round(6)}")
    # Compare 2 answer is it the same
    if np.allclose(tau_rtb,calculate_tau):
        return "The answer is the same"
    else :
        return "Wrong!"

# Random q and w
q = np.random.rand(3)*np.pi
w = np.random.rand(6)
print (f"Random q is {q}")
print("------------------------------------------------------")
print(prooftau(q,w))


Random q is [2.60152814 0.99958337 1.3225545 ]
------------------------------------------------------
Calculate tau is [ 1.089804 -0.291349 -0.162635]
RTB tau is [ 1.089804 -0.291349 -0.162636]
The answer is the same
